In [ ]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample


warnings.filterwarnings("ignore")

#import pingouin as pg
from sklearn.utils import resample

In [ ]:
from statannot import add_stat_annotation
import seaborn as sns


In [ ]:
BAGs_data = pd.read_excel('data/BAG_sex.xlsx')

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

df = BAGs_data.copy()


Q1 = df['GAP_corrected'].quantile(0.25)
Q3 = df['GAP_corrected'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 2.5 * IQR
upper_bound = Q3 + 2.5 * IQR



df.reset_index(inplace = True, drop = True)


vars_physical = ['gender_equal_l', 'Migration', 'Polution_conc_inv', 'Eq']

vars_extended = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

scaler = MinMaxScaler()
df[vars_physical] = scaler.fit_transform(df[vars_physical])
df['physical'] = df[vars_physical].mean(axis=1)

df[vars_extended] = scaler.fit_transform(df[vars_extended])
df['extended'] = df[vars_extended].mean(axis=1)

total_exposomes = vars_physical + vars_extended
df[total_exposomes] = scaler.fit_transform(df[total_exposomes])
df['total_exposomes'] = df[total_exposomes].mean(axis=1)

expos_ = ['gender_equal_l', 'Migration', 'Polution_conc_inv', 'Eq', 'representation_l', 'free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est', 'physical', 'extended', 'total_exposomes']

metrics_df = pd.DataFrame(columns=['slope','r_value', 'd_cohen', 'Sex', 'Exposome'])

for i in range(10):
    for exp in expos_:
    
        
        df_exp_ = df.copy()
    
        df_exp_.dropna(inplace = True)
        df_exp_.reset_index(inplace = True, drop = True)
    
    
        X_train, X_test, y_train, y_test = train_test_split(df_exp_[['GAP_corrected', 'Sex']], df_exp_[exp], test_size=0.1, stratify = df_exp_['Sex'], random_state=i)
    
        df_exp_ = pd.concat([X_test, y_test], axis =1)
        df_exp_.reset_index(inplace = True, drop = True)
    
        df_exp_f = df_exp_[df_exp_.Sex == 'F']
        df_exp_f = df_exp_f.copy()
        df_exp_f.reset_index(inplace = True, drop = True)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(df_exp_f[exp], df_exp_f['GAP_corrected'])
        confidence_interval = 2.58 * std_err  # 95% CI: slope ± 1.96 * std_err
        slope_ci_lower = slope - confidence_interval
        slope_ci_upper = slope + confidence_interval
    
    
        r_value1 = np.abs(r_value)
        d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
    
        current_metrics_df = pd.DataFrame(columns=['intercept', 'std_err', 'slope','r_value', 'd_cohen', 'Sex', 'Exposome'])
        current_metrics_df.loc[i, :] = [intercept, std_err, slope, r_value, d_cohen, 'F', exp]
        metrics_df = pd.concat([metrics_df, current_metrics_df])
    
        df_exp_m = df_exp_[df_exp_.Sex == 'M']
        df_exp_m = df_exp_m.copy()
        df_exp_m.reset_index(inplace = True, drop = True)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(df_exp_m[exp], df_exp_m['GAP_corrected'])
        confidence_interval = 2.58 * std_err  # 95% CI: slope ± 1.96 * std_err
        slope_ci_lower = slope - confidence_interval
        slope_ci_upper = slope + confidence_interval
    
    
        r_value1 = np.abs(r_value)
        d_cohen = (2 * r_value1) / np.sqrt(1 - r_value1**2)
    
        current_metrics_df = pd.DataFrame(columns=['intercept', 'std_err', 'slope','r_value', 'd_cohen', 'Sex', 'Exposome'])
        current_metrics_df.loc[i, :] = [intercept, std_err, slope, r_value, d_cohen, 'M', exp]
        metrics_df = pd.concat([metrics_df, current_metrics_df])

metrics_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

expos_ = ['total_exposomes', 'physical', 'extended',
          'gender_equal_l', 'Migration', 'Eq', 'Polution_conc_inv', 
          'representation_l', 'free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est',
           ]

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(8, 10))  
axes = axes.flatten()  

for idx, exp in enumerate(expos_):
    if idx >= len(axes):  
        break
    
    cm = metrics_df[metrics_df['Exposome'] == exp]
    cm["slope"] = pd.to_numeric(cm["slope"], errors="coerce")
    
    females = cm[cm['Sex'] == 'F']['slope']
    males = cm[cm['Sex'] == 'M']['slope']

    p_normal_f = stats.shapiro(females)[1] if len(females) > 3 else 0
    p_normal_m = stats.shapiro(males)[1] if len(males) > 3 else 0

    if p_normal_f > 0.05 and p_normal_m > 0.05:
        stat, p_value = stats.ttest_ind(females, males, equal_var=False)
        test_used = "T-test"
    else:
        stat, p_value = stats.mannwhitneyu(females, males)
        test_used = "Mann-Whitney U"

    stat, p_value = stats.ttest_ind(females, males, equal_var=False)

    custom_palette = ['khaki',  
                      'mediumpurple']  

    sns.violinplot(ax=axes[idx], data=cm, x="Sex", y="slope", palette=custom_palette)

    axes[idx].set_title(f"{exp}\n t = {stat:.2f} p={p_value:.2e}")
    axes[idx].set_ylabel("slope")
    axes[idx].set_xlabel("Sex")
    axes[idx].set_ylim(-8, -1)

plt.tight_layout()

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from scipy.stats import linregress



fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(8, 11))  
axes = axes.flatten()  

for idx, exp in enumerate(expos_):
    if idx >= len(axes):
        break

    
    cm = metrics_df[metrics_df['Exposome'] == exp].copy() 
    cm["slope"] = pd.to_numeric(cm["slope"], errors="coerce") 
    cm["intercept"] = pd.to_numeric(cm["intercept"], errors="coerce") 

    
    
    females = cm[cm['Sex'] == 'F']
    males = cm[cm['Sex'] == 'M']

    titles = f"{exp}\n F: s = {females.slope.mean():.2f} d={females.d_cohen.mean():.2f}\n M: s = {males.slope.mean():.2f} d={males.d_cohen.mean():.2f}"

    ax = axes[idx]


    x_vals = np.linspace(0, 1, 100)
    
    y_females = females["intercept"].mean() + females["slope"].mean() * x_vals
    ax.plot(x_vals, y_females, color="darkgoldenrod", linestyle="--", linewidth=0.5, label="F")

    y_males = males["intercept"].mean() + males["slope"].mean() * x_vals
    ax.plot(x_vals, y_males, color="indigo", linestyle="--", linewidth=0.5, label="M")

    std_err_f = females["slope"].std() / np.sqrt(len(females))
    std_err_m = males["slope"].std() / np.sqrt(len(males))

    alpha = 0.01  # 99% CI
    t_critical = stats.t.ppf(1 - alpha / 2, df=len(cm) - 2)  # Valor crítico de t
    
    
    ci_f = t_critical * std_err_f
    ci_m = t_critical * std_err_m

    ci_upper_f = y_females + ci_f
    ci_lower_f = y_females - ci_f

    ci_upper_m = y_males + ci_m
    ci_lower_m = y_males - ci_m


    ax.fill_between(x_vals, ci_lower_f, ci_upper_f, color="olive", alpha=0.2)
    ax.fill_between(x_vals, ci_lower_m, ci_upper_m, color="blueviolet", alpha=0.2)


    ax.set_title(titles)
    ax.set_ylabel("Slope")
    ax.legend()
    ax.set_ylim([-10, 10])

plt.tight_layout()


plt.show()
